## Trying to make sure saving function works

In [1]:
%%time
ndays =2

date_list = [start_date + timedelta(days=x) for x in range(ndays)]
#target_files = [f'{datadir}_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # list target files
target_files = [f'{datadir}/LLC4320_pre-SWOT_ACC_SMST_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # lis
# chunk size ... aiming for ~100 MB chunks
# these chunks seem to work OK for up to ~20 day simulations, but more 
# testing is needed to figure out optimal parameters for longer simulations
#original
#tchunk = 6 
#xchunk = 150
#ychunk = 150

# drop the vector variables if loading derived variables because we are going to load the rotated ones in the next cell
#if sampling_details['DERIVED_VARIABLES']:
 #   drop_variables={'U', 'V', 'oceTAUX', 'oceTAUY'}
#else:
 #   drop_variables={}

ds = xr.open_mfdataset(target_files, 
                       #parallel=True, 
                       #drop_variables=drop_variables,
                      #chunks={'i':xchunk, 'j':ychunk, 'time':tchunk}
                      )

# XC, YC and Z are the same at all times, so select a single time
# (note, this breaks for a single file - always load >1 file)
X = ds.XC.isel(time=0) 
Y = ds.YC.isel(time=0)




# load the corresponding derived fields (includes steric height, vorticity, and transformed vector variables for current and wind stress)
if sampling_details['DERIVED_VARIABLES']:
    derivedir = datadir + 'derived/'
    derived_files = [f'{derivedir}LLC4320_pre-SWOT_{RegionName}_derived-fields_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # list target files
    dsd = xr.open_mfdataset(derived_files)
    
    # merge the derived and raw data
    ds = ds.merge(dsd)
    # rename the transformed vector variables to their original names
    ds = ds.rename_vars({'U_transformed':'U', 'V_transformed':'V', 
                         'oceTAUX_transformed':'oceTAUX', 'oceTAUY_transformed':'oceTAUY'})


# drop a bunch of other vars we don't actually use - can comment this out if these are wanted
ds = ds.drop_vars({'DXV','DYU', 'DXC','DXG', 'DYC','DYG', 'XC_bnds', 'YC_bnds', 'Zp1', 'Zu','Zl','Z_bnds', 'nb'})
#ds



#%%time
#del sys.modules['osse_tools_Copy1'] 
#from osse_tools_Copy1 import download_llc4320_data, compute_derived_fields, get_survey_track, survey_interp

survey_track, survey_indices, sampling_parameters = get_survey_track(ds, sampling_details)

# print specified sampling_details + any default values
#sampling_parameters




# ---- generate name of file to save outputs in ---- 
filename_base = (f'OSSE_{RegionName}_{sampling_details["SAMPLING_STRATEGY"]}_{start_date}_to_{start_date + timedelta(ndays)}_maxdepth{int(sampling_parameters["zrange"][1])}')
filename_out_base = (f'{outputdir}{filename_base}')
#print(filename_base)


#%%time
%matplotlib inline
plt.figure(figsize=(15,5))

# map of Theta at time zero
ax = plt.subplot(1,2,1)
ssto = plt.pcolormesh(X,Y,ds.Theta.isel(k=0, time=0).values, shading='auto')
if not (sampling_details['SAMPLING_STRATEGY'] == 'mooring' or sampling_details['SAMPLING_STRATEGY'] == 'sim_mooring'):
    tracko = plt.scatter(survey_track.lon, survey_track.lat, c=(survey_track.time-survey_track.time[0])/1e9/86400, cmap='Reds', s=0.75)
    plt.colorbar(ssto).set_label('SST, $^o$C')
    plt.colorbar(tracko).set_label('days from start')
    plt.title('SST and survey track: ' + RegionName + ', '+ sampling_details['SAMPLING_STRATEGY'])
else:
    plt.plot(survey_track.lon, survey_track.lat, marker='*', c='r')
    plt.title('SST and mooring location: ' + RegionName + ' region, ' + sampling_details['SAMPLING_STRATEGY'] )


# depth/time plot of first few datapoints
ax = plt.subplot(1,2,2)
iplot = slice(0,20000)
if not (sampling_details['SAMPLING_STRATEGY'] == 'mooring' or sampling_details['SAMPLING_STRATEGY'] == 'sim_mooring'):
    plt.plot(survey_track.time.isel(points=iplot), survey_track.dep.isel(points=iplot), marker='.')
else:
    # not quite right but good enough for now.
    # (times shouldn't increase with depth)
    plt.scatter((np.tile(survey_track['time'].isel(time=iplot), int(survey_track['dep'].data.size))),
         np.tile(survey_track['dep'], int(survey_track['time'].isel(time=iplot).data.size)),marker='.')             
#plt.xlim([start_date + datetime.timedelta(days=0), start_date + datetime.timedelta(days=2)])
plt.ylabel('Depth, m')
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.gcf().autofmt_xdate()
plt.title(f"Sampling pattern, hspeed ={sampling_parameters['hspeed']}, vspeed ={sampling_parameters['vspeed']}")


# save
#if SAVE_FIGURES:
#    plt.savefig('/data2/Dropbox/projects/adac/figures/' + filename_base + '_sampling.png', dpi=400, transparent=False, facecolor='white')

plt.show()




#%%time

#del sys.modules['osse_tools'] 
#from osse_tools import survey_interp, get_survey_track

subsampled_data, sgridded = survey_interp(ds, survey_track, survey_indices, sampling_parameters)
sgridded

#%%time
# 3d fields
vbls3d = ['Theta','Salt','U','V','vorticity']
vbls3d = ['Theta','Salt']
ylim = [min(sgridded['depth'].values), max(sgridded['depth'].values)]
ylim = [-200, -1]

nr = len(vbls3d) # # of rows
fig,ax=plt.subplots(nr,figsize=(8,len(vbls3d)*2),constrained_layout=True)


for j in range(nr):
    sgridded[vbls3d[j]].plot(ax=ax[j], ylim=ylim)
    ax[j].plot(sgridded.time.data, -sgridded.KPPhbl.data, c='k')
    ax[j].set_title(vbls3d[j])

#if SAVE_FIGURES:
    #plt.savefig(figdir + filename_base + '_3D.png', dpi=400, transparent=False, facecolor='white')

NameError: name 'start_date' is not defined